In [ ]:
!python3 -m pip install "torch" "transformers==4.27.1" "diffusers[torch]==0.19.3"
!python3 -m pip uninstall accelerate -y
!python3 -m pip install -U onediff

In [ ]:
!python3 -m pip install --pre oneflow -f https://oneflow-staging.oss-cn-beijing.aliyuncs.com/branch/master/cu117

In [ ]:
!python3 -m pip install huggingface_hub

In [ ]:
!huggingface-cli login --token <Hf_TOKEN>

In [ ]:
import torch
import oneflow as flow
from pathlib import Path
from onediff.infer_compiler import oneflow_compile
from diffusers import StableDiffusionXLPipeline

In [ ]:
# def handler(job):
#     job_input = job["input"]
#     pretrained_model_name_or_path = job_input["pretrained_model_name_or_path"]
#     prompt = job_input["prompt"]
#     saved_image = job_input["saved_output_image"]
#     file_name = job_input["compiled_file_name"]
#     job_output = {}

#     try:
#         if job_input["mode"] == "compile":
#             # Compilation Mode
#             pipe = StableDiffusionXLPipeline.from_pretrained(
#                 pretrained_model_name_or_path,
#                 torch_dtype=torch.float16,
#                 variant="fp16",
#                 use_safetensors=True,
#             )
#             pipe.to("cuda")

#             pipe.unet = oneflow_compile(pipe.unet)
#             if Path(file_name).exists():
#                 print(f"Loading the compiled graph from {file_name}...")
#                 pipe.unet.warmup_with_load(file_name)
#             # Save the compiled graph to network storage
#             if not Path(file_name).exists():
#                 pipe.unet.save_graph(file_name)
#                 print(f"Saved the compiled graph to {file_name}.")
#         elif job_input["mode"] == "compile_inference":
#             # Compile Inference Mode
#             pipe = StableDiffusionXLPipeline.from_pretrained(
#                 pretrained_model_name_or_path,
#                 torch_dtype=torch.float16,
#                 variant="fp16",
#                 use_safetensors=True,
#             )
#             pipe.to("cuda")

#             # Read the compiled graph from network storage
#             if Path(file_name).exists():
#                 print(f"Loading the compiled graph from {file_name}...")
#                 pipe.unet.warmup_with_load(file_name)

#             # Perform inference
#             image = pipe(prompt).images[0]
#             print(f"Saved the image to {saved_image}.")

#     except Exception as e:
#         print(f"An error occurred: {e}")


In [ ]:
def handler(job):
    job_input = job["input"]

    pretrained_model_name_or_path = job_input["pretrained_model_name_or_path"]
    prompt = job_input["prompt"]
    saved_image = job_input["saved_output_image"]
    file_name = job_input["compiled_file_name"]
    job_output = {}

    try:
        pipe = StableDiffusionXLPipeline.from_pretrained(
            pretrained_model_name_or_path,
            torch_dtype=torch.float16,
            variant="fp16",
            use_safetensors=True,
        )
        pipe.to("cuda")

        pipe.unet = oneflow_compile(pipe.unet)
        if Path(file_name).exists():
            print(f"Loading the compiled graph from {file_name}...")
            pipe.unet.warmup_with_load(file_name)

        image = pipe(prompt).images[0]
        print(f"Saved the image to {saved_image}.")
        image.save(saved_image)

        if not Path(file_name).exists():
            pipe.unet.save_graph(file_name)
            print(f"Saved the compiled graph to {file_name}.")

    except Exception as e:
        print(f"An error occurred: {e}")


In [ ]:
#runpod.serverless.start({"handler": handler}) 
# Example :  Sample
import json

# Define the variables
pretrained_model_name_or_path = "stabilityai/stable-diffusion-xl-base-1.0"
prompt = "photo of two cats fighting"
saved_image = "sdxl-base-out-.png"
file_name = "unet_compile"

# Create a dictionary to store the variables
data = {
    "pretrained_model_name_or_path": pretrained_model_name_or_path,
    "prompt": prompt,
    "saved_output_image": saved_image,
    "compiled_file_name": file_name,
    "mode": "compile_inference"
}

# Convert the dictionary to a JSON object
json_object = json.dumps(data, indent=4)


handler({"input": data})